<center><h1>Kan_Herun_HW4</h1></center>
<br>
<br>

Name: Herun Kan
<br>
Github Username: herunkan
<br>
USC ID: 7222919427

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.linear_model as lm
import statsmodels.api as sm
import sklearn.preprocessing as pp
import os
import random

Get the AReM Data Set

In [2]:
col_names = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13','var_rss13', 'avg_rss23', 'var_rss23']

In [3]:
bending1_folder = '../data/AReM/bending1'

bending1_data = {}
bending1_folderdata = os.listdir(bending1_folder)
bending1_folderdata.sort()

for i, file in enumerate(bending1_folderdata):
    file_path = os.path.join(bending1_folder, file)
    bending1_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #bending1_data[i]['Activity'] = 'bending1'


In [4]:
bending2_folder = '../data/AReM/bending2'

bending2_data = {}
bending2_folderdata = os.listdir(bending2_folder)
bending2_folderdata.sort()

for i, file in enumerate(bending2_folderdata):
    file_path = os.path.join(bending2_folder, file)
    bending2_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #bending2_data[i]['Activity'] = 'bending2'

#bending dataset4 caused some problem
bending2_data[3] = pd.read_csv('../data/AReM/bending2/dataset4.csv', skiprows= 5, names = col_names, sep=' ', index_col=False)


In [5]:
# Had ParseError, found out it was trailing comma in a couple csv's so manually deleted those

cycling_folder = '../data/AReM/cycling'

cycling_data = {}
cycling_folderdata = os.listdir(cycling_folder)
cycling_folderdata.sort()

for i, file in enumerate(cycling_folderdata):
    file_path = os.path.join(cycling_folder, file)
    cycling_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #cycling_data[i]['Activity'] = 'cycling'

In [6]:
lying_folder = '../data/AReM/lying'

lying_data = {}
lying_folderdata = os.listdir(lying_folder)
lying_folderdata.sort()

for i, file in enumerate(lying_folderdata):
    file_path = os.path.join(lying_folder, file)
    lying_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #lying_data[i]['Activity'] = 'lying'

In [7]:
sitting_folder = '../data/AReM/sitting'

sitting_data = {}
sitting_folderdata = os.listdir(sitting_folder)
sitting_folderdata.sort()

for i, file in enumerate(sitting_folderdata):
    file_path = os.path.join(sitting_folder, file)
    sitting_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #sitting_data[i]['Activity'] = 'sitting'

In [8]:
standing_folder = '../data/AReM/standing'

standing_data = {}
standing_folderdata = os.listdir(standing_folder)
standing_folderdata.sort()

for i, file in enumerate(standing_folderdata):
    file_path = os.path.join(standing_folder, file)
    standing_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #standing_data[i]['Activity'] = 'standing'

In [9]:
walking_folder = '../data/AReM/walking'

walking_data = {}
walking_folderdata = os.listdir(walking_folder)
walking_folderdata.sort()

for i, file in enumerate(walking_folderdata):
    file_path = os.path.join(walking_folder, file)
    walking_data[i] = pd.read_csv(file_path, skiprows= 5, names = col_names)
    #walking_data[i]['Activity'] = 'walking'

In [10]:
all_data ={**bending1_data, **bending2_data, **cycling_data, **lying_data, **sitting_data, **standing_data, **walking_data} 

len(all_data)

15

### (b) Test and Train Data

In [11]:
def test_train_split(data, test_num):
    train_data = {k: v for k, v in data.items() if k not in test_num}
    test_data = {k: v for k, v in data.items() if k in test_num}
    
    return train_data, test_data

test_num_bending = [0, 1]  
test_num_other = [0, 1, 2] 

bending1_train, bending1_test = test_train_split(bending1_data, test_num_bending)
bending2_train, bending2_test = test_train_split(bending2_data, test_num_bending)
cycling_train, cycling_test = test_train_split(cycling_data, test_num_other)
lying_train, lying_test = test_train_split(lying_data, test_num_other)
sitting_train, sitting_test = test_train_split(sitting_data, test_num_other)
standing_train, standing_test = test_train_split(standing_data, test_num_other)
walking_train, walking_test = test_train_split(walking_data, test_num_other)


### (c) Feature Extraction

#### i. Research

"Classification of time series usually needs extracting features from them. So, extract the time-domain features such as minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all the 6 time series in each instance. Estimate the standard deviation of each of the time-domain features, 90% bootsrap confidence interval for the standard deviation of each feature to select the three most important time-domain features" (Surbhi Jain, Github, https://github.com/SurbhiJainUSC/Time-Series-Classification).

#### ii. Extraction

In [12]:

activities = {
    'bending1': bending1_data,
    'bending2': bending2_data,
    'cycling': cycling_data,
    'lying': lying_data,
    'sitting': sitting_data,
    'standing': standing_data,
    'walking': walking_data
}

feature_df = []

def extract_features(df):
    features = {}
    for column in ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']:
        features[f'min_{column}'] = df[column].min()
        features[f'max_{column}'] = df[column].max()
        features[f'mean_{column}'] = df[column].mean()
        features[f'median_{column}'] = df[column].median()
        features[f'std_{column}'] = df[column].std()
        features[f'1st_quart_{column}'] = df[column].quantile(0.25)
        features[f'3rd_quart_{column}'] = df[column].quantile(0.75)
    return pd.Series(features)

for activity, data in activities.items():
    for key, df in data.items():
        feature_values = extract_features(df)
        feature_values.name = f"{activity}_{key}" 
        feature_df.append(feature_values)

combined_features = pd.DataFrame(feature_df)
combined_features

,min_avg_rss12,max_avg_rss12,mean_avg_rss12,median_avg_rss12,std_avg_rss12,1st_quart_avg_rss12,3rd_quart_avg_rss12,min_var_rss12,max_var_rss12,mean_var_rss12,...,std_avg_rss23,1st_quart_avg_rss23,3rd_quart_avg_rss23,min_var_rss23,max_var_rss23,mean_var_rss23,median_var_rss23,std_var_rss23,1st_quart_var_rss23,3rd_quart_var_rss23
bending1_0,37.25,45.00,40.624792,40.500,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.00,1.3000
bending1_1,38.00,45.67,42.812812,42.500,1.435550,42.00,43.6700,0.0,1.22,0.372437,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.00,1.3000
bending1_2,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
bending1_3,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
bending1_4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
walking_10,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
walking_11,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
walking_12,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200
walking_13,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600


In [13]:
#Helped by Chatgpt

#Prompt:
#how does bootstrap work in pandas?

#Response:
# Bootstrapping is a resampling technique used to estimate statistics (like the mean, median, variance, confidence intervals, etc.) of a population from a sample dataset. It is particularly useful when you want to quantify the uncertainty of a statistic without making strong parametric assumptions about the distribution of the data.

# Bootstrapping in Pandas

# In pandas, you can implement bootstrapping directly using methods like sample(), which allows you to sample data from a DataFrame or Series. Here’s a simple example illustrating how to bootstrap the mean of a pandas Series:

# import pandas as pd
# import numpy as np

# # Sample data
# data = pd.Series([1, 2, 3, 4, 5])

# # Number of bootstrap samples
# n_boot = 1000
# boot_means = []

# # Bootstrapping process
# for _ in range(n_boot):
#     # Generate a bootstrap sample
#     sample = data.sample(frac=1, replace=True)  # frac=1 means to sample the same size as the original data
#     boot_means.append(sample.mean())  # Calculate the mean for this bootstrap sample

# # Calculate the 90% confidence interval
# lower_ci = np.percentile(boot_means, 5)  # 5th percentile
# upper_ci = np.percentile(boot_means, 95)  # 95th percentile

# # Display the results
# print("Bootstrap Mean CI (90%):", (lower_ci, upper_ci))


def bootstrap_std(data, n_boot=1000, alpha=0.1):
    bootstrapped_stds = []
    n = len(data)
    for _ in range(n_boot):
        sample = np.random.choice(data, size=n, replace=True)
        bootstrapped_stds.append(np.std(sample, ddof=1))

    lower_ci = np.percentile(bootstrapped_stds, 100 * (alpha / 2))
    upper_ci = np.percentile(bootstrapped_stds, 100 * (1 - alpha / 2))
    
    return np.std(data, ddof=1), lower_ci, upper_ci

std_dev_ci = {}

for column in combined_features.columns:
    data = combined_features[column]
    sd, ci_lower, ci_upper = bootstrap_std(data, n_boot=1000, alpha=0.1)
    std_dev_ci[column] = {
        'sd': sd,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    }

# Convert the results into a DataFrame for easier viewing
std_dev_ci_df = pd.DataFrame(std_dev_ci).T
std_dev_ci_df.columns = ['Standard Deviation', 'Lower CI', 'Upper CI']

# Display the results
print(std_dev_ci_df)

                     Standard Deviation  Lower CI   Upper CI
min_avg_rss12                  9.569975  8.225518  10.746853
max_avg_rss12                  4.394362  3.302244   5.302611
mean_avg_rss12                 5.335718  4.708118   5.883051
median_avg_rss12               5.440054  4.836325   6.004812
std_avg_rss12                  1.772153  1.575165   1.944887
1st_quart_avg_rss12            6.153590  5.591746   6.609438
3rd_quart_avg_rss12            5.138925  4.354701   5.835581
min_var_rss12                  0.000000  0.000000   0.000000
max_var_rss12                  5.062729  4.624133   5.431565
mean_var_rss12                 1.574164  1.407543   1.706575
median_var_rss12               1.412244  1.235842   1.542657
std_var_rss12                  0.884105  0.799859   0.943430
1st_quart_var_rss12            0.946386  0.834879   1.031225
3rd_quart_var_rss12            2.125266  1.898283   2.294371
min_avg_rss13                  2.956462  2.769063   3.100501
max_avg_rss13           

i will go with mean, median and standard deviation

## 2. Time Series Classification Part 2: Binary and Multiclass Classification

### (a) Binary Classification Using Logistic Regression

#### i. Plots

#### ii. Splitted Plots

#### iii. Time Series Segments

#### iv. Confusion Matrix

#### v. Test Classifier

#### vi. Separation

#### vii. Imbalance

### (b) Binary Classification Using L1-penalized logistic regression

#### i. Time Series Segments

#### ii. Comparison

### (c) Multi-class Classification (The Realistic Case)

#### i. Time Series Segments

#### ii. Naive Bayes

#### ii. Comparison

## 3. ISLR 4.8.3

## 4. ISLR 4.8.7

## 5.1. Extra Practice ISLR 4.8.4

## 5.2. Extra Practice ISLR 4.8.9